# Optimización de un modelo de Random Forest

Este notebook recoge los resultados de la búsqueda del mejor modelo de clasificación mediante Random Forest. El método entrena varios árboles de decisión (de clasificación en este caso) en submuestras de los datos y combina sus resultados para mejorar su precisión.

Para buscar el mejor modelo posible, se tratará de buscar los mejores hiperparámetros para:

* El número de árboles del bosque.
* La profundidad máxima que alcanzan estos.
* Función para evaluar una nueva división de una rama.

### Preparación de los datos

In [3]:
# Estructuras de datos
import pandas as pd
import numpy as np

# Librerías de optimización de hiperparámetros
import optuna

# Accuracy
from sklearn.metrics import accuracy_score

# Modelo
from sklearn.ensemble import RandomForestClassifier

# Cargar los datos
from data_and_submissions import *

# Métodos para los entrenamientos con CV
from train_cv_methods import *

Vamos a usar la siguiente partición de los datos:

* 60% train $\sim$ 50 datos
* 20% validation $\sim$ 18 datos (se define al aplicar cross-validación en el ajuste)
* 20% test $\sim$ 18 datos

In [2]:
X_train, X_test, y_train, y_test, test_kaggle = load_data()
print("Tamaño del dataset de train:", X_train.shape)
print("Tamaño del dataset de test:", X_test.shape)

Tamaño del dataset de train: (68, 410)
Tamaño del dataset de test: (18, 410)


### Modelo

Primera prueba de resultados, probando con valores concretos para el número de árboles y la profundidad de los mismos (parámetros ``n_estimators`` y ``max_depth`` respectivamente) y utilizando el método ``GridSearchCV`` de ``sklearn`` para realizar la búsqueda:

In [4]:
model_RF = RandomForestClassifier(random_state=0)
param_grid_RF = {
    "n_estimators": range(50, 1050, 50),
    "criterion": ["gini", "entropy"],
    "max_depth": range(1, 21)
}

In [4]:
# Definir y entrenar el modelo
cv_results_RF = train_GridSearchCV(model_RF, param_grid_RF, X_train, X_test, y_train, y_test)
top_acc = top_acc_GridSearchCV(cv_results_RF["mean_test_score"])
models_same_acc_GridSearchCV(cv_results_RF, top_acc)

[{'criterion': 'entropy', 'max_depth': 4, 'n_estimators': 600},
 {'criterion': 'entropy', 'max_depth': 5, 'n_estimators': 650},
 {'criterion': 'entropy', 'max_depth': 5, 'n_estimators': 750},
 {'criterion': 'entropy', 'max_depth': 5, 'n_estimators': 800},
 {'criterion': 'entropy', 'max_depth': 6, 'n_estimators': 750},
 {'criterion': 'entropy', 'max_depth': 6, 'n_estimators': 800},
 {'criterion': 'entropy', 'max_depth': 7, 'n_estimators': 750},
 {'criterion': 'entropy', 'max_depth': 7, 'n_estimators': 800},
 {'criterion': 'entropy', 'max_depth': 8, 'n_estimators': 750},
 {'criterion': 'entropy', 'max_depth': 8, 'n_estimators': 800},
 {'criterion': 'entropy', 'max_depth': 9, 'n_estimators': 750},
 {'criterion': 'entropy', 'max_depth': 9, 'n_estimators': 800},
 {'criterion': 'entropy', 'max_depth': 10, 'n_estimators': 750},
 {'criterion': 'entropy', 'max_depth': 10, 'n_estimators': 800},
 {'criterion': 'entropy', 'max_depth': 11, 'n_estimators': 750},
 {'criterion': 'entropy', 'max_depth'

In [5]:
# Definir y entrenar el modelo
model_RF_opt = RandomForestClassifier(criterion="entropy", max_depth=20, n_estimators=800, random_state=0)
model_RF_opt.fit(X_train, y_train)

# Predicción en partición de test
y_pred_RF = model_RF_opt.predict(X_test)

# Precisión en partición de test
accuracy = accuracy_score(y_test, y_pred_RF)
print("Accuracy: {:0.2f}%".format(accuracy * 100))

Accuracy: 88.89%


La librería ``optuna`` es un framework específico para la optimización de hiperparámetros, repetiremos el proceso de búsqueda anterior utilizando esta librería en base a 2 métodos de búsqueda de hiperparámetros:

* **GridSampler:** equivalente a la anterior búsqueda de grid de sklearn. Lo usaremos para que los resultados sean comparables.
* **TPE:** algoritmo para hacer una "búsqueda inteligente" de hiperparámetros. Debería ahorrar intentos de combinaciones haciendo una selección inteligente de las pruebas. En nuestro caso le permitiremos probar un 10% del número de combinaciones posibles. 

In [8]:
def objectiveRF_Grid(trial):
    '''
    Define la función a optimizar por medio de un sampler de tipo GridSampler.
    En este caso se trata de maximizar el accuracy
    '''
    n_estimators =  trial.suggest_int("n_estimators", 50, 1000)
    criterion = trial.suggest_categorical("criterion", ["gini", "entropy"])
    max_depth = trial.suggest_int("max_depth", 1, 20)
    
    modelRF_optuna = RandomForestClassifier(criterion = criterion, max_depth = max_depth, n_estimators = n_estimators, 
                                            random_state=0)
    
    modelRF_optuna.fit(X_train, y_train)

    y_pred_RF_optuna = modelRF_optuna.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred_RF_optuna)
    return accuracy

In [9]:
# Prueba con GridSampler
optuna.logging.set_verbosity(optuna.logging.WARNING)

search_space = {
    "n_estimators": range(50, 1001, 50), 
    "criterion": ["gini", "entropy"], 
    "max_depth": range(1, 21)
}
sampler = optuna.samplers.GridSampler(search_space)
study_Grid = optuna.create_study(direction="maximize", sampler=sampler)
study_Grid.optimize(objectiveRF_Grid)

In [10]:
study_Grid.best_trial

FrozenTrial(number=7, values=[0.8888888888888888], datetime_start=datetime.datetime(2022, 6, 11, 23, 12, 10, 710916), datetime_complete=datetime.datetime(2022, 6, 11, 23, 12, 13, 897665), params={'n_estimators': 900, 'criterion': 'entropy', 'max_depth': 7}, distributions={'n_estimators': IntUniformDistribution(high=1000, low=50, step=1), 'criterion': CategoricalDistribution(choices=('gini', 'entropy')), 'max_depth': IntUniformDistribution(high=20, low=1, step=1)}, user_attrs={}, system_attrs={'search_space': OrderedDict([('criterion', ['entropy', 'gini']), ('max_depth', [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]), ('n_estimators', [50, 100, 150, 200, 250, 300, 350, 400, 450, 500, 550, 600, 650, 700, 750, 800, 850, 900, 950, 1000])]), 'grid_id': 137}, intermediate_values={}, trial_id=7, state=TrialState.COMPLETE, value=None)

In [6]:
# Definir y entrenar el modelo
modelRF_optuna_Grid = RandomForestClassifier(criterion="entropy", max_depth=7, n_estimators=900, random_state=0)  
modelRF_optuna_Grid.fit(X_train, y_train)

# Predicción en partición de test
y_pred_RF_optuna_Grid = modelRF_optuna_Grid.predict(X_test)

# Precisión en partición de test
accuracy = accuracy_score(y_test, y_pred_RF_optuna_Grid)
print("Accuracy: {:0.2f}%".format(accuracy * 100))

Accuracy: 88.89%


El método GridSampler no permite fijar semilla por lo que los resultados que devuelve serán diferentes en cuanto a parámetros aunque en cualquier caso tendrán la misma precisión en la partición de test.

Para probar TPE es necesario redefinir la función objetivo sobre la que se crea el estudio, ya que GridSampler definía dentro de la función el espacio en el que se tomaban los parámetros mientras que los valores exactos a probar los define fuera de la misma. Sin embargo, TPE define en la misma función objetivo los valores a tomar, por lo que hay que incluir los pasos.

In [7]:
def objectiveRF_TPE(trial):
    '''
    Define la función a optimizar por medio de un sampler de tipo TPE.
    En este caso se trata de maximizar el accuracy
    '''
    n_estimators =  trial.suggest_int("n_estimators", 50, 1000, 50) # optuna incluye en el rango el máximo y el mínimo
    criterion = trial.suggest_categorical("criterion", ["gini", "entropy"])
    max_depth = trial.suggest_int("max_depth", 1, 20)
    
    modelRF_optuna = RandomForestClassifier(criterion = criterion, max_depth = max_depth, n_estimators = n_estimators, 
                                            random_state=0)
    
    modelRF_optuna.fit(X_train, y_train)

    y_pred_RF_optuna = modelRF_optuna.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred_RF_optuna)
    return accuracy

In [8]:
# Prueba con TPE
optuna.logging.set_verbosity(optuna.logging.WARNING)

sampler = optuna.samplers.TPESampler(seed=0)  # Asegurar los reproducibilidad de los resultados
study_TPE = optuna.create_study(direction="maximize", sampler=sampler)
study_TPE.optimize(objectiveRF_TPE, n_trials=80)
# n_trials = (20 x 2 x 20) * 0.1 = 80

In [9]:
study_TPE.best_trial

FrozenTrial(number=2, values=[0.8888888888888888], datetime_start=datetime.datetime(2022, 7, 2, 11, 25, 47, 456791), datetime_complete=datetime.datetime(2022, 7, 2, 11, 25, 50, 390511), params={'n_estimators': 1000, 'criterion': 'entropy', 'max_depth': 11}, distributions={'n_estimators': IntUniformDistribution(high=1000, low=50, step=50), 'criterion': CategoricalDistribution(choices=('gini', 'entropy')), 'max_depth': IntUniformDistribution(high=20, low=1, step=1)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=2, state=TrialState.COMPLETE, value=None)

In [7]:
# Definir y entrenar el modelo
modelRF_optuna_TPE = RandomForestClassifier(criterion="entropy", max_depth=11, n_estimators=1000, random_state=0)  
modelRF_optuna_TPE.fit(X_train, y_train)

# Predicción en partición de test
y_pred_RF_optuna_TPE = modelRF_optuna_TPE.predict(X_test)

# Precisión en partición de test
accuracy = accuracy_score(y_test, y_pred_RF_optuna_TPE)
print("Accuracy: {:0.2f}%".format(accuracy * 100))

Accuracy: 88.89%


La librería ``optuna`` ha permitido obtener mejores resultados.

El código anterior, aunque realiza una búsqueda sobre el mismo rango de parámetros usados para el segundo intento con ``sklearn``, no está aplicando cross-validation en el entrenamiento. La siguiente celda sí lo implementa mediante el método ``OptunaSearchCV`` de ``optuna``:

In [11]:
optuna.logging.set_verbosity(optuna.logging.WARNING)

# Definir y entrenar el modelo
model_RF = RandomForestClassifier(random_state=0)
param_grid_RF = {
    "n_estimators": optuna.distributions.IntUniformDistribution(50, 1000, 50),
    "criterion": optuna.distributions.CategoricalDistribution(["gini", "entropy"]),
    "max_depth": optuna.distributions.IntUniformDistribution(1, 20)
}

optuna_search = optuna.integration.OptunaSearchCV(model_RF, param_grid_RF, cv=4, n_trials=800, random_state=0)
# n_trials = 20 x 2 x 20 = 800
optuna_search.fit(X_train, y_train)

C:\Users\saral\AppData\Local\Temp\ipykernel_13484\2766480869.py:11: ExperimentalWarning: OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.
  optuna_search = optuna.integration.OptunaSearchCV(model_RF, param_grid_RF, cv=4, n_trials=800, random_state=0)


OptunaSearchCV(cv=4, estimator=RandomForestClassifier(random_state=0),
               n_trials=800,
               param_distributions={'criterion': CategoricalDistribution(choices=('gini', 'entropy')),
                                    'max_depth': IntUniformDistribution(high=20, low=1, step=1),
                                    'n_estimators': IntUniformDistribution(high=1000, low=50, step=50)},
               random_state=0)

In [13]:
top_acc = top_acc_OptunaSearchCV(optuna_search.trials_)
models_same_acc_OptunaSearchCV(optuna_search.trials_, top_acc)

[{'n_estimators': 750, 'criterion': 'entropy', 'max_depth': 12},
 {'n_estimators': 750, 'criterion': 'entropy', 'max_depth': 6},
 {'n_estimators': 750, 'criterion': 'entropy', 'max_depth': 7},
 {'n_estimators': 750, 'criterion': 'entropy', 'max_depth': 13},
 {'n_estimators': 800, 'criterion': 'entropy', 'max_depth': 6},
 {'n_estimators': 650, 'criterion': 'entropy', 'max_depth': 5},
 {'n_estimators': 800, 'criterion': 'entropy', 'max_depth': 14},
 {'n_estimators': 650, 'criterion': 'entropy', 'max_depth': 5},
 {'n_estimators': 750, 'criterion': 'entropy', 'max_depth': 8},
 {'n_estimators': 800, 'criterion': 'entropy', 'max_depth': 6},
 {'n_estimators': 800, 'criterion': 'entropy', 'max_depth': 8},
 {'n_estimators': 800, 'criterion': 'entropy', 'max_depth': 10},
 {'n_estimators': 750, 'criterion': 'entropy', 'max_depth': 12},
 {'n_estimators': 800, 'criterion': 'entropy', 'max_depth': 14},
 {'n_estimators': 800, 'criterion': 'entropy', 'max_depth': 8},
 {'n_estimators': 750, 'criterion'

In [8]:
# Definir y entrenar el modelo
optunaCV_opt = RandomForestClassifier(n_estimators=800, criterion="entropy", max_depth=20, random_state=0)
optunaCV_opt.fit(X_train, y_train)

# Predicción en partición de test
y_pred_RF_optunaCV = optunaCV_opt.predict(X_test)

# Precisión en partición de test
accuracy = accuracy_score(y_test, y_pred_RF_optunaCV)
print("Accuracy: {:0.2f}%".format(accuracy * 100))

Accuracy: 88.89%


La precisión obtenida mediante los 4 modelos es la misma. Lo siguiente es comprobar si las predicciones de cada modelo son idénticas o si esta coincidencia se debe al mismo número de aciertos pero en diferentes muestras.

In [9]:
y_pred_model1 = model_RF_opt.predict(test_kaggle)
y_pred_model2 = modelRF_optuna_Grid.predict(test_kaggle)
y_pred_model3 = modelRF_optuna_TPE.predict(test_kaggle)
y_pred_model4 = optunaCV_opt.predict(test_kaggle)

In [10]:
results = {"GridSearchCV": y_pred_model1, "Optuna & GridSampler": y_pred_model2, 
           "Optuna & TPE": y_pred_model3, "OptunaSearchCV": y_pred_model4}

results_df = pd.DataFrame(results)    
results_df["All the same"] = results_df.eq(results_df.iloc[:, 0], axis=0).all(1)
results_df[results_df["All the same"] == False]

,GridSearchCV,Optuna & GridSampler,Optuna & TPE,OptunaSearchCV,All the same
39,1,1,0,1,False
58,1,0,0,1,False
76,1,0,1,1,False
79,1,1,0,1,False
86,1,0,0,1,False
...,...,...,...,...,...
119532,0,0,1,0,False
119541,0,1,1,0,False
119560,1,0,0,1,False
119610,1,0,0,1,False


Las primeras 2 filas ya muestran que los modelos que no se basan en cross-validación (2 y 3) no hacen las mismas predicciones.

Comprobación adicional para los dos restantes:

In [11]:
results2 = {"GridSearchCV": y_pred_model1, "OptunaSearchCV": y_pred_model4}

results_df2 = pd.DataFrame(results2)    
results_df2["All the same"] = results_df2.eq(results_df2.iloc[:, 0], axis=0).all(1)
results_df2[results_df2["All the same"] == False]

,GridSearchCV,OptunaSearchCV,All the same


Los dos modelos de cross-validación si hacen predicciones idénticas, vamos a probar por tanto el rendimiento en Kaggle de 3 versiones distintas de predicciones.

In [12]:
create_submission(y_pred_model1, "RF_opt_CV")

(119748, 2)


In [13]:
create_submission(y_pred_model2, "RF_opt_GridSampler")

(119748, 2)


In [14]:
create_submission(y_pred_model3, "RF_opt_TPE")

(119748, 2)
